In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
%pip install crewai_tools langchain_groq  langchain_community cohere --quiet
!pip install kagglehub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of embedchain[github,youtube] to determin

In [ ]:
!pip install 'crewai[tools]'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: crewai-tools
    Found existing installation: crewai-tools 0.1.6
    Uninstalling crewai-tools-0.1.6:
      Successfully uninstalled crewai-tools-0.1.6


In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
from getpass import getpass
from crewai import Agent, Task, Crew, Process
from textwrap import dedent
# from langchain_openai import ChatOpenAI
# ↑ uncomment to use OpenAI's API
from langchain_groq import ChatGroq
# ↑ uncomment to use Groq's API
os.environ["GROQ_API_KEY"] = getpass("Enter GROQ_API_KEY: ")

Enter GROQ_API_KEY: ··········


In [ ]:
import kagglehub
import os
import shutil

# Download the dataset from Kaggle
dataset_path = kagglehub.dataset_download("tobiasbueck/multilingual-customer-support-tickets")

print("Path to dataset files:", dataset_path)

# Specify the folder path where you want to save the dataset
folder_path = 'dataset'

# Create the folder if it doesn't exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Check if the dataset path is a directory
if os.path.isdir(dataset_path):
    # If it's a directory, copy all files to the target folder
    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            src_file_path = os.path.join(root, file)
            dest_file_path = os.path.join(folder_path, file)
            shutil.copy(src_file_path, dest_file_path)
    print(f"Dataset files copied to: {folder_path}")
else:
    # If it's a file, assume it's an archive and try to extract it
    try:
        shutil.unpack_archive(dataset_path, folder_path)
        print(f"Dataset extracted to: {folder_path}")
    except shutil.ReadError:
        print(f"The dataset format is not recognized. Please check the contents of: {dataset_path}")

# List files in the dataset folder
print("Files in the dataset folder:")
for root, dirs, files in os.walk(folder_path):
    for file in files:
        print(os.path.join(root, file))

100%|██████████| 7.05M/7.05M [00:00<00:00, 86.5MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/tobiasbueck/multilingual-customer-support-tickets/versions/9
Dataset files copied to: dataset
Files in the dataset folder:
dataset/dataset-tickets-multi-lang-4-20k.csv
dataset/dataset-tickets-multi-lang3-4k.csv


In [ ]:
!pip install langchain_huggingface, langchain_community

ERROR: Invalid requirement: 'langchain_huggingface,': Expected end or semicolon (after name and no valid version specifier)
    langchain_huggingface,
                         ^


In [ ]:

from crewai_tools import JSONSearchTool
json_search_tool = JSONSearchTool(json_path="faq.json",
                                   config=dict(
        llm=dict(
            provider="groq", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="llama-3.3-70b-versatile",
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="huggingface", # or openai, ollama, ...
            config=dict(
                model="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
                # task_type="retrieval_document",
                #task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    ))

ImportError: cannot import name 'from_env' from 'langchain_core.utils' (/usr/local/lib/python3.11/dist-packages/langchain_core/utils/__init__.py)

In [ ]:
from crewai_tools import ScrapeWebsiteTool
docs_scrape_tool = ScrapeWebsiteTool(
    website_url=""
)

In [ ]:

js=json_search_tool.run

In [ ]:
js

<bound method BaseTool.run of JSONSearchTool(name="Search a JSON's content", description="A tool that can be used to semantic search a query the FAQ.json JSON's content.", args_schema=<class 'crewai_tools.tools.json_search_tool.json_search_tool.FixedJSONSearchToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x7ea9aa20f240>, summarize=False, adapter=EmbedchainAdapter(embedchain_app=<embedchain.app.App object at 0x7ea9a964a310>, summarize=False), config=None)>

In [ ]:
faq_path = "FAQ.json"
csv_path = "dataset/dataset-tickets-multi-lang-4-20k.csv"

if os.path.exists(faq_path) and os.path.exists(csv_path):
    print("✅ Dataset files are present and ready for use.")
else:
    print("❌ Missing dataset files! Check the Kaggle download process.")


✅ Dataset files are present and ready for use.


In [ ]:
# Perform a test scrape
scraped_content = docs_scrape_tool.run()

Using Tool: Read website content


In [ ]:
from crewai import Agent, Task, Crew, Process
from langchain_groq import ChatGroq
from crewai_tools import CSVSearchTool
csv_search_tool = CSVSearchTool("dataset/dataset-tickets-multi-lang3-4k.csv")

Inserting batches in chromadb:   0%|          | 0/19 [00:00<?, ?it/s]


In [ ]:
csv_search_tool

CSVSearchTool(name="Search a CSV's content", description="A tool that can be used to semantic search a query the dataset/dataset-tickets-multi-lang3-4k.csv CSV's content.", args_schema=<class 'crewai_tools.tools.csv_search_tool.csv_search_tool.FixedCSVSearchToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x7ea9aa20f240>, summarize=False, adapter=EmbedchainAdapter(embedchain_app=<embedchain.app.App object at 0x7ea9a8758150>, summarize=False), config=None)

In [ ]:
agent_1 = Agent(
    role=dedent(
        """
        Customer Support IT Specialist
        """
    ),
    backstory=dedent(
        """
        You are a highly skilled Customer Support IT Specialist at customer.io, a leading IT company providing services such as IT consulting, cybersecurity, cloud solutions, and software development.
        You are an AI agent trained on a comprehensive dataset of customer support IT tickets.
        Your expertise lies in understanding and resolving technical issues related to IT support,
        such as software troubleshooting, hardware problems, network issues, and more.
        You have been fine-tuned to provide accurate and helpful responses to users' technical queries.
        """
    ),
    goal=dedent(
        """
        Provide precise and accurate answers to users' technical questions based on the customer support IT dataset.
        Your goal is to ensure users receive the best possible support and solutions to their IT-related issues.
        Respond in a friendly and conversational tone, as if you are speaking directly to the customer.
        Do not make assumptions—always verify the information before responding.
        """
    ),
    tools=[csv_search_tool],
    allow_delegation=False,
    verbose=True,
    llm=ChatGroq(temperature=0.5, model_name="deepseek-r1-distill-qwen-32b"),
)
agent_2 = Agent(
    role=dedent(
        """
        Customer Support General-IT
        """
    ),
    backstory=dedent(
        """
        You are a highly skilled Customer Support IT Specialist at customer.io, a leading IT company providing services such as IT consulting, cybersecurity, cloud solutions, and software development.
        You assist customers with technical issues, provide guidance on the company's services, and ensure a smooth support experience.
        Your expertise spans troubleshooting, system integrations, DevOps support, and IT infrastructure scaling.
        """
    ),
    goal=dedent(
        """
        - Provide accurate and helpful answers to customer queries about customer.io services.
        - Troubleshoot technical issues effectively and provide solutions.
        - Guide users on cybersecurity best practices and data recovery options.
        - Assist customers with software development, cloud migration, and IT infrastructure scaling.
        - Ensure customer satisfaction by delivering timely and precise responses.
        - Respond in a friendly and conversational tone, as if you are speaking directly to the customer.
        - Do not make assumptions—always verify the information before responding.
        """
    ),
    tools=[json_search_tool],
    allow_delegation=False,
    verbose=True,
    llm=ChatGroq(temperature=0.5, model_name="deepseek-r1-distill-qwen-32b"),
)
agent_3 = Agent(
    role="Website Support Representative",
    goal=dedent(
        """
        Provide friendly, accurate, and complete customer support by referencing information available on the BITSWITS website.
        Respond in a conversational tone, as if you are speaking directly to the customer.
        Do not make assumptions—every response should be backed by website content.
        """
    ),
    backstory=dedent(
        """
        You are a dedicated support representative for customer.io (, responsible for assisting customers with their inquiries by using
        verified information from the company's website. Your goal is to provide clear, comprehensive, and helpful responses, ensuring
        a positive customer experience. You do not make assumptions—every response should be backed by website content.
        """
    ),
    allow_delegation=False,
    verbose=True,
    tools=[docs_scrape_tool],
    llm=ChatGroq(temperature=0.5, model_name="deepseek-r1-distill-qwen-32b"),
)

In [ ]:
import datetime

task_1 = Task(
    description=dedent("""
        Review the customer's inquiry and provide a detailed, conversational response.
        Ensure that the answer is comprehensive, accurate, and adheres to the high-quality standards expected for customer support.
        Respond as if you are speaking directly to the customer, using a friendly and helpful tone.
        Verify that all parts of the customer's inquiry have been addressed thoroughly.
        Do not make assumptions—always verify the information before responding.
    """),
    expected_output=dedent("""
        A friendly and conversational response to the customer's inquiry, addressing all concerns and providing clear, actionable solutions.
        The response should be well-structured and supported by verified information from the dataset.
    """),
    agent=agent_1,
    tools=[csv_search_tool]
)

task_2 = Task(
    description=dedent("""
        Handle customer support inquiries for customer.io by responding to common questions, troubleshooting technical issues,
        and providing guidance on IT solutions such as cloud migration, cybersecurity, software development, and system integration.
        Use the FAQ knowledge base to ensure accurate and helpful responses.
        Respond in a conversational tone, as if you are speaking directly to the customer.
        Do not make assumptions—always verify the information before responding.
    """),
    expected_output=dedent("""
        - Accurate and prompt responses to customer queries based on the customer.io FAQ.
        - Effective troubleshooting steps for reported technical issues.
        - Clear guidance on customer.io services, including software development, IT consulting, and cybersecurity solutions.
        - A friendly and conversational tone in all communications.
    """),
    agent=agent_2,
    tools=[json_search_tool]
)

task_3 = Task(
    description=dedent("""
        Assist customers by answering their inquiries using information from the Customer.io website.
        Provide accurate, friendly, and detailed responses without making assumptions.
        Respond in a conversational tone, as if you are speaking directly to the customer.
        Ensure that customers receive complete and helpful support based on available website content.
    """),
    expected_output=dedent("""
        - Accurate and complete responses to customer inquiries using website information.
        - A friendly and professional tone in all communications.
        - Clear guidance on Customer.io services, policies, and features as per website details.
        - No assumptions—only verified details from the website should be shared.
    """),
    agent=agent_3,
    tools=[docs_scrape_tool]
)




In [ ]:
result = task_3.execute()
print(result)



> Entering new CrewAgentExecutor chain...
<think>
Okay, so I need to figure out how to help the customer with their inquiry using only the information from the Customer.io website. Let me start by understanding what the user is asking. They mentioned that they're a Website Support Representative for BITSWITS and need to assist customers using the website's content. 

First, I should read the website content to gather accurate information. I'll use the Read website content tool to access the necessary details. Once I have the information, I'll structure my response in a friendly and professional tone, making sure not to assume anything beyond what's provided on the website.

I need to ensure that every response is backed by the website content, so I'll double-check each point I include. The response should be comprehensive, covering all aspects of the inquiry without adding any personal opinions or assumptions. 

After gathering the information, I'll organize it in a clear and convers

In [ ]:
crew = Crew(
        agents=[agent_1,agent_2,agent_3],
        tasks=[task_1,task_2,task_3],
        verbose=False,  # You can set it to True or False
        manager_llm=ChatGroq(temperature=0.3, model_name="qwen-2.5-32b"),
        # ↑ indicates the verbosity level for logging during execution.
        process=Process.hierarchical
        # ↑ the process flow that the crew will follow (e.g., sequential, hierarchical).
    )

In [ ]:
inputs = {
    "customer": "Goutam Sachdev",
    "inquiry": "I'm unable to log in to company website from my work computer. What could be causing this issue, and how can I fix it?"
}
result = crew.kickoff(inputs=inputs)



> Entering new CrewAgentExecutor chain...
Thought: To ensure that the response to the customer's inquiry is comprehensive, accurate, and adheres to high-quality standards, I need to first understand the specifics of the customer's inquiry. I will start by asking the Customer Support General-IT for a summary of the customer's inquiry and any relevant details.

Action: Ask question to co-worker
Action Input: {"coworker": "Customer Support General-IT", "question": "Could you provide a summary of the customer's inquiry and any relevant details? This is to ensure we can provide a comprehensive, accurate, and high-quality response.", "context": "To prepare a detailed, conversational response that addresses all concerns and provides clear, actionable solutions, we need to understand the customer's inquiry fully."}


> Entering new CrewAgentExecutor chain...
<think>
Okay, so I need to figure out how to respond to the user's request. They mentioned that I did something wrong with the format, 

In [ ]:

from IPython.display import display, Markdown

markdown_text = result  # Adjust this based on the actual attribute

# Display the markdown content
display(Markdown(markdown_text))

Hey there! I'm sorry to hear you're having trouble logging into customer.io from your work computer. Let's try to troubleshoot this step by step. 

1. **Check Network Issues**: First, ensure that your work network isn't blocking access to customer.io. You can try accessing the site from a different network, like your phone's hotspot, to test this.

2. **Account Lockout**: If you've entered the wrong password multiple times, your account might be locked. Look out for any messages about account lockouts and try resetting your password using the recovery options provided.

3. **MFA Issues**: If you're using multi-factor authentication (MFA), make sure your authenticator app is working correctly. If you're having trouble with MFA, you can contact support to bypass it temporarily.

4. **Account Status**: Check if your account is active. If you're unsure, reach out to your administrator to confirm your account status.

5. **Clear Browser Cache**: Sometimes, clearing your browser's cache and cookies can resolve login issues. Try doing this and then attempt to log in again.

6. **Alternative Devices**: If the problem persists, try logging in from a different device, like your smartphone or another computer, to see if the issue is specific to your work computer.

If none of these steps work, please don't hesitate to reach out to customer.io support. We're here to help and can look into the issue further. Let me know if you need any clarification or additional assistance!